In [14]:
import os
import cv2
from skimage.feature import graycomatrix, graycoprops
from scipy.spatial.distance import euclidean

# Function to extract GLCM features
def glcm_features(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    glcm = graycomatrix(gray_image, [1], [0], 256, symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast').flatten()
    correlation = graycoprops(glcm, 'correlation').flatten()
    energy = graycoprops(glcm, 'energy').flatten()
    homogeneity = graycoprops(glcm, 'homogeneity').flatten()
    return contrast, correlation, energy, homogeneity

# Function to calculate Euclidean distance between two feature vectors
def calculate_similarity(feature1, feature2):
    distance = 0
    for f1, f2 in zip(feature1, feature2):
        distance += (f1 - f2)**2
    return distance**0.5

# Path to folders
root_folder = "./PlantVillage/Color_Images/"
healthy_folder = "Apple___healthy"
diseased_folders = ["Apple_scab", "Apple_black_rot", "Apple_rust"]

# Iterate over each healthy leaf image
for healthy_image_file in os.listdir(os.path.join(root_folder, healthy_folder)):
    healthy_image_path = os.path.join(root_folder, healthy_folder, healthy_image_file)
    if os.path.isfile(healthy_image_path):
        healthy_image = cv2.imread(healthy_image_path)
        if healthy_image is not None:  # Check if image is loaded successfully
            healthy_features = glcm_features(healthy_image)
            
            # Iterate over each folder containing diseased leaf images
            for diseased_folder in diseased_folders:
                best_similarity = float('inf')
                best_diseased_leaf_image = None
                for diseased_image_file in os.listdir(os.path.join(root_folder, diseased_folder)):
                    diseased_image_path = os.path.join(root_folder, diseased_folder, diseased_image_file)
                    if os.path.isfile(diseased_image_path):
                        diseased_image = cv2.imread(diseased_image_path)
                        if diseased_image is not None:  # Check if image is loaded successfully
                            diseased_features = glcm_features(diseased_image)
                            similarity = calculate_similarity(healthy_features, diseased_features)
                            
                            # Update best similarity and best diseased leaf image
                            if similarity < best_similarity:
                                best_similarity = similarity
                                best_diseased_leaf_image = diseased_image_path
                
                if best_diseased_leaf_image is not None:
                    print(f"For healthy leaf image {healthy_image_file}, most similar diseased leaf image in folder {diseased_folder} is {best_diseased_leaf_image} with similarity score {best_similarity}")
                else:
                    print(f"No suitable diseased leaf image found for healthy leaf image {healthy_image_file} in folder {diseased_folder}")
        else:
            print(f"Error loading healthy leaf image: {healthy_image_path}")
    else:
        print(f"Invalid file: {healthy_image_path}")

For healthy leaf image img119.JPG, most similar diseased leaf image in folder Apple_scab is ./PlantVillage/Color_Images/Apple_scab/img268.JPG with similarity score [6.2248292]
For healthy leaf image img119.JPG, most similar diseased leaf image in folder Apple_black_rot is ./PlantVillage/Color_Images/Apple_black_rot/img10.JPG with similarity score [178.56370735]
For healthy leaf image img119.JPG, most similar diseased leaf image in folder Apple_rust is ./PlantVillage/Color_Images/Apple_rust/img172.JPG with similarity score [8.34986878]
For healthy leaf image img131.JPG, most similar diseased leaf image in folder Apple_scab is ./PlantVillage/Color_Images/Apple_scab/img530.JPG with similarity score [515.57701707]
For healthy leaf image img131.JPG, most similar diseased leaf image in folder Apple_black_rot is ./PlantVillage/Color_Images/Apple_black_rot/img103.JPG with similarity score [32.20093857]
For healthy leaf image img131.JPG, most similar diseased leaf image in folder Apple_rust is 